In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
import re
import os

class WanliuHousingCrawler:
    def __init__(self):
        self.driver = webdriver.Chrome()
        self.wait = WebDriverWait(self.driver, 15)
        
    def crawl_second_hand_houses(self):
        """爬取万柳二手房数据"""
        print("开始爬取万柳二手房数据...")
        url = "https://esf.fang.com/house-a015277-b02313/"
        self.driver.get(url)
        time.sleep(2)
        
        all_data = []
        max_pages = 20  # 限制爬取页数
        
        for page in range(max_pages):
            print(f"正在爬取二手房第{page+1}页...")
            
            try:
                # 等待房源列表加载 - 使用更通用的选择器
                self.wait.until(
                    EC.presence_of_element_located((By.XPATH, "//dl[contains(@class, 'clearfix') and @dataflag='bg']"))
                )
                
                # 获取所有房源 - 使用正确的选择器
                houses = self.driver.find_elements(By.XPATH, "//dl[contains(@class, 'clearfix') and @dataflag='bg']")
                print(f"找到 {len(houses)} 个房源")
                
                for i, house in enumerate(houses):
                    try:
                        print(f"正在处理第 {i+1} 个房源...")
                        
                        # 提取面积信息 - 使用 tel_shop
                        try:
                            area_element = house.find_element(By.CLASS_NAME, "tel_shop")
                            area_text = area_element.text
                            # 从文本中提取面积数字
                            area_match = re.search(r'(\d+\.?\d*)\s*㎡', area_text)
                            if area_match:
                                area = float(area_match.group(1))
                                print(f"提取到面积: {area}㎡")
                            else:
                                print(f"无法从文本中提取面积: {area_text}")
                                continue
                        except Exception as e:
                            print(f"提取面积时出错: {e}")
                            continue
                        
                        # 提取价格信息
                        try:
                            price_element = house.find_element(By.CLASS_NAME, "price_right")
                            price_span = price_element.find_element(By.CLASS_NAME, "red")
                            price_b = price_span.find_element(By.TAG_NAME, "b")
                            price_text = price_b.text
                            
                            # 清理价格数据
                            price_match = re.search(r'(\d+\.?\d*)', price_text)
                            if price_match:
                                price = float(price_match.group(1))
                                print(f"提取到价格: {price}万")
                            else:
                                print(f"无法解析价格: {price_text}")
                                continue
                        except Exception as e:
                            print(f"提取价格时出错: {e}")
                            continue
                            
                        house_data = {
                            '面积(㎡)': area,
                            '价格(万)': price
                        }
                        all_data.append(house_data)
                        print(f"成功提取: {area}㎡, {price}万")
                        
                    except Exception as e:
                        print(f"提取第{i+1}个房源信息时出错: {e}")
                        continue
                
                # 翻页功能
                try:
                    # 查找包含"下一页"文本的链接
                    next_page = self.driver.find_element(By.XPATH, "//a[contains(text(), '下一页')]")
                    next_page.click()
                    time.sleep(1)
                except NoSuchElementException:
                    print("没有找到下一页按钮，已到达最后一页")
                    break
                    
            except Exception as e:
                print(f"爬取第{page+1}页时出错: {e}")
                # 打印当前页面的一些信息帮助调试
                print("当前URL:", self.driver.current_url)
                print("页面标题:", self.driver.title)
                break
        
        return pd.DataFrame(all_data)
    
    def crawl_rental_houses(self):
        """爬取万柳租房数据"""
        print("开始爬取万柳租房数据...")
        url = "https://zu.fang.com/house-a015277-b02313/"
        self.driver.get(url)
        time.sleep(3)
        
        all_data = []
        max_pages = 20  # 限制爬取页数
        
        for page in range(max_pages):
            print(f"正在爬取租房第{page+1}页...")
            
            try:
                # 等待房源列表加载
                self.wait.until(
                    EC.presence_of_element_located((By.CLASS_NAME, "houseList"))
                )
                
                # 获取所有房源
                houses = self.driver.find_elements(By.CLASS_NAME, "list")
                print(f"找到 {len(houses)} 个租房房源")
                
                for i, house in enumerate(houses):
                    try:
                        print(f"正在处理第 {i+1} 个租房房源...")
                        
                        # 提取面积信息
                        try:
                            area_element = house.find_element(By.CLASS_NAME, "font15")
                            area_text = area_element.text
                            # 从文本中提取面积数字
                            area_match = re.search(r'(\d+\.?\d*)\s*㎡', area_text)
                            if area_match:
                                area = float(area_match.group(1))
                                print(f"提取到租房面积: {area}㎡")
                            else:
                                print(f"无法从文本中提取租房面积: {area_text}")
                                continue
                        except Exception as e:
                            print(f"提取租房面积时出错: {e}")
                            continue
                        
                        # 提取价格信息 - 修正：使用price类
                        try:
                            # 查找包含"moreInfo"的div，然后找到其中的price类
                            more_info = house.find_element(By.CLASS_NAME, "moreInfo")
                            price_element = more_info.find_element(By.CLASS_NAME, "price")
                            price_text = price_element.text
                            
                            # 清理价格数据
                            price_match = re.search(r'(\d+\.?\d*)', price_text)
                            if price_match:
                                price = float(price_match.group(1))
                                print(f"提取到租房价格: {price}元/月")
                            else:
                                print(f"无法解析租房价格: {price_text}")
                                continue
                        except Exception as e:
                            print(f"提取租房价格时出错: {e}")
                            # 如果上面的方法失败，尝试直接查找price类
                            try:
                                price_elements = house.find_elements(By.CLASS_NAME, "price")
                                if price_elements:
                                    price_text = price_elements[0].text
                                    price_match = re.search(r'(\d+\.?\d*)', price_text)
                                    if price_match:
                                        price = float(price_match.group(1))
                                        print(f"提取到租房价格(备用方法): {price}元/月")
                                    else:
                                        continue
                                else:
                                    continue
                            except:
                                continue
                            
                        house_data = {
                            '面积(㎡)': area,
                            '租金(元/月)': price
                        }
                        all_data.append(house_data)
                        print(f"成功提取租房: {area}㎡, {price}元/月")
                        
                    except Exception as e:
                        print(f"提取第{i+1}个租房房源信息时出错: {e}")
                        continue
                
                # 翻页功能
                try:
                    # 查找包含"下一页"文本的链接
                    next_page = self.driver.find_element(By.XPATH, "//a[contains(text(), '下一页')]")
                    next_page.click()
                    time.sleep(1)
                except NoSuchElementException:
                    print("没有找到下一页按钮，已到达最后一页")
                    break
                    
            except Exception as e:
                print(f"爬取第{page+1}页时出错: {e}")
                break
        
        return pd.DataFrame(all_data)
    
    def analyze_data(self, df, data_type="二手房"):
        """数据分析"""
        print(f"\n=== {data_type}数据统计 ===")
        print(f"总记录数: {len(df)}")
        
        if '价格(万)' in df.columns:
            print(f"平均价格: {df['价格(万)'].mean():.2f}万")
            print(f"价格中位数: {df['价格(万)'].median():.2f}万")
            print(f"最高价格: {df['价格(万)'].max():.2f}万")
            print(f"最低价格: {df['价格(万)'].min():.2f}万")
            
            # 计算单价
            df['单价(元/㎡)'] = (df['价格(万)'] * 10000 / df['面积(㎡)']).round(2)
            print(f"平均单价: {df['单价(元/㎡)'].mean():.2f}元/㎡")
        
        elif '租金(元/月)' in df.columns:
            print(f"平均租金: {df['租金(元/月)'].mean():.2f}元/月")
            print(f"租金中位数: {df['租金(元/月)'].median():.2f}元/月")
            print(f"最高租金: {df['租金(元/月)'].max():.2f}元/月")
            print(f"最低租金: {df['租金(元/月)'].min():.2f}元/月")
            
            # 计算租售比
            df['租金单价(元/㎡/月)'] = (df['租金(元/月)'] / df['面积(㎡)']).round(2)
            print(f"平均租金单价: {df['租金单价(元/㎡/月)'].mean():.2f}元/㎡/月")
        
        print(f"平均面积: {df['面积(㎡)'].mean():.2f}㎡")
        print(f"面积中位数: {df['面积(㎡)'].median():.2f}㎡")
    
    def save_to_excel(self, df, filename):
        """保存数据到Excel"""
        # 获取当前工作目录
        current_dir = os.getcwd()
        file_path = os.path.join(current_dir, filename)
        
        df.to_excel(file_path, index=False)
        print(f"数据已保存到: {file_path}")
    
    def close(self):
        """关闭浏览器"""
        self.driver.quit()
        print("浏览器已关闭")

# 主程序
def main():
    crawler = WanliuHousingCrawler()
    
    try:
        # 爬取二手房数据
        second_hand_df = crawler.crawl_second_hand_houses()
        if not second_hand_df.empty:
            crawler.analyze_data(second_hand_df, "万柳二手房")
            crawler.save_to_excel(second_hand_df, "万柳二手房数据.xlsx")
            print("\n万柳二手房数据样本:")
            print(second_hand_df.head())
        else:
            print("未获取到二手房数据")
        
        # 爬取租房数据
        rental_df = crawler.crawl_rental_houses()
        if not rental_df.empty:
            crawler.analyze_data(rental_df, "万柳租房")
            crawler.save_to_excel(rental_df, "万柳租房数据.xlsx")
            print("\n万柳租房数据样本:")
            print(rental_df.head())
        else:
            print("未获取到租房数据")
            
    except Exception as e:
        print(f"程序执行出错: {e}")
    finally:
        crawler.close()

if __name__ == "__main__":
    main()

开始爬取万柳二手房数据...
正在爬取二手房第1页...
找到 60 个房源
正在处理第 1 个房源...
提取到面积: 395.0㎡
提取到价格: 8500.0万
成功提取: 395.0㎡, 8500.0万
正在处理第 2 个房源...
提取到面积: 420.21㎡
提取到价格: 7000.0万
成功提取: 420.21㎡, 7000.0万
正在处理第 3 个房源...
提取到面积: 114.59㎡
提取到价格: 2400.0万
成功提取: 114.59㎡, 2400.0万
正在处理第 4 个房源...
提取到面积: 305.08㎡
提取到价格: 4300.0万
成功提取: 305.08㎡, 4300.0万
正在处理第 5 个房源...
提取到面积: 150.21㎡
提取到价格: 2100.0万
成功提取: 150.21㎡, 2100.0万
正在处理第 6 个房源...
提取到面积: 266.79㎡
提取到价格: 5900.0万
成功提取: 266.79㎡, 5900.0万
正在处理第 7 个房源...
提取到面积: 286.01㎡
提取到价格: 5120.0万
成功提取: 286.01㎡, 5120.0万
正在处理第 8 个房源...
提取到面积: 147.15㎡
提取到价格: 2150.0万
成功提取: 147.15㎡, 2150.0万
正在处理第 9 个房源...
提取到面积: 395.0㎡
提取到价格: 8950.0万
成功提取: 395.0㎡, 8950.0万
正在处理第 10 个房源...
提取到面积: 355.0㎡
提取到价格: 6500.0万
成功提取: 355.0㎡, 6500.0万
正在处理第 11 个房源...
提取到面积: 265.0㎡
提取到价格: 5150.0万
成功提取: 265.0㎡, 5150.0万
正在处理第 12 个房源...
提取到面积: 235.84㎡
提取到价格: 3400.0万
成功提取: 235.84㎡, 3400.0万
正在处理第 13 个房源...
提取到面积: 316.0㎡
提取到价格: 5400.0万
成功提取: 316.0㎡, 5400.0万
正在处理第 14 个房源...
提取到面积: 464.87㎡
提取到价格: 6850.0万
成功提取: 464.87㎡, 6850.0万
正在处理第 15 个房源..